<a href="https://colab.research.google.com/github/cvcdrew/WeatherWise-Andrew/blob/main/andrew_Untitled26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 🧪 Install necessary packages
!pip install -q requests matplotlib pandas seaborn ipywidgets

# ✅ Imports
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output
import re

# ✅ API Setup
os.environ['OPENWEATHERMAP_API_KEY'] = "7cf20335110caaf78db0fecb31852d45"
API_KEY = os.environ['OPENWEATHERMAP_API_KEY']
BASE_URL = "https://api.openweathermap.org/data/2.5/forecast"
INTERVALS_PER_DAY = 8

# ✅ Robust AI-style natural language parser with improved city extraction
def parse_weather_question(question):
    question = question.strip()
    original = question
    question_lower = question.lower()

    # Default values
    condition = "rain"
    city = None
    day = "today"

    # Extract condition keywords
    condition_match = re.search(r"\b(rain|snow|clear|cloudy)\b", question_lower)
    if condition_match:
        condition = condition_match.group(1)

    # Extract day keywords (check longer phrase first)
    if "day after tomorrow" in question_lower:
        day = "day_after_tomorrow"
    elif "tomorrow" in question_lower:
        day = "tomorrow"

    # Extract city name carefully from original question (case-sensitive)
    city_match = re.search(
        r"\bin ([A-Za-zÀ-ÿ\s]+?)(?:\?|$| tomorrow| day after tomorrow| will| is| does| rain| snow| clear| cloudy)",
        original, re.IGNORECASE)
    if city_match:
        city = city_match.group(1).strip()
    else:
        capitalized = re.findall(r"\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)*", original)
        if capitalized:
            city = capitalized[-1]
        else:
            city = "London"

    fixes = {
        "paris": "Paris,FR",
        "london": "London,GB",
        "new york": "New York,US",
        "sydney": "Sydney,AU"
    }

    city_key = city.lower()
    city = fixes.get(city_key, city.title())

    return {"condition": condition, "city": city, "day": day}

# ✅ Weather data fetch
def get_weather_data(location: str, forecast_days: int = 5, units: str = "metric") -> dict:
    params = {
        "q": location,
        "cnt": forecast_days * INTERVALS_PER_DAY,
        "units": units,
        "appid": API_KEY
    }
    res = requests.get(BASE_URL, params=params)
    if res.status_code != 200:
        raise Exception(f"Error fetching weather: {res.status_code} - {res.text}")
    return res.json()

# ✅ Generate AI response based on parsed question
def generate_weather_response(parsed):
    if not parsed:
        return "⚠️ Sorry, I couldn't understand your question. Try asking like: 'Will it rain in Paris tomorrow?'"

    condition = parsed["condition"]
    city = parsed["city"]
    day = parsed["day"]

    display_city = city.split(",")[0]

    try:
        data = get_weather_data(city)
        target_date = datetime.utcnow().date()
        if day == "tomorrow":
            target_date += timedelta(days=1)
        elif day == "day_after_tomorrow":
            target_date += timedelta(days=2)

        for entry in data["list"]:
            entry_date = datetime.strptime(entry["dt_txt"], "%Y-%m-%d %H:%M:%S").date()
            weather_desc = entry["weather"][0]["description"].lower()
            if entry_date == target_date and condition in weather_desc:
                return f"✅ Yes, it looks like it will be {condition} in {display_city} {day.replace('_', ' ')}."
        return f"❌ No, it doesn't look like it will be {condition} in {display_city} {day.replace('_', ' ')}."

    except Exception as e:
        return f"❌ Could not retrieve weather for {display_city}: {e}"

# ✅ Preprocessing for plotting
def preprocess_weather_data(dates, values):
    df = pd.DataFrame({"Date": pd.to_datetime(dates), "Value": values})
    return df

# ✅ Visualizations without emojis in titles and without 'Noto Color Emoji' font
def create_temperature_plot(weather_data):
    plt.rcParams['font.family'] = ['DejaVu Sans', 'Liberation Sans', 'sans-serif']
    dates = [entry["dt_txt"] for entry in weather_data["list"]]
    temps = [entry["main"]["temp"] for entry in weather_data["list"]]
    df = preprocess_weather_data(dates, temps)
    plt.figure(figsize=(10, 5))
    sns.lineplot(data=df, x="Date", y="Value", marker="o")
    plt.title("Temperature Forecast (UTC)")  # Emoji removed
    plt.xlabel("Date/Time")
    plt.ylabel("Temperature (°C)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def create_precipitation_plot(weather_data):
    plt.rcParams['font.family'] = ['DejaVu Sans', 'Liberation Sans', 'sans-serif']
    dates = [entry["dt_txt"] for entry in weather_data["list"]]
    precip = [entry.get("rain", {}).get("3h", 0) for entry in weather_data["list"]]
    df = preprocess_weather_data(dates, precip)
    plt.figure(figsize=(10, 5))
    sns.barplot(data=df, x="Date", y="Value", color="skyblue")
    plt.title("Precipitation Forecast (UTC)")  # Emoji removed
    plt.xlabel("Date/Time")
    plt.ylabel("Precipitation (mm)")
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()

# ✅ Widgets
location_box = widgets.Text(description="City:", placeholder="e.g. London", value="London")
days_slider = widgets.IntSlider(description="Days:", min=1, max=5, value=3)
units_dropdown = widgets.Dropdown(options=["metric", "imperial", "standard"], value="metric", description="Units:")
vis_toggle = widgets.ToggleButtons(options=["Temperature", "Precipitation", "Both"], description="Visual:")
question_box = widgets.Text(description="Ask AI:", placeholder="e.g. Will it rain in Paris tomorrow?")
go_button = widgets.Button(description="Show Forecast", button_style="primary")
ask_ai_button = widgets.Button(description="Ask AI", button_style="info")
output = widgets.Output()

# ✅ Click handler for Show Forecast button
def on_click(b):
    with output:
        clear_output()
        try:
            if question_box.value.strip():
                parsed = parse_weather_question(question_box.value)
                print(generate_weather_response(parsed))
                location = parsed["city"]
                day = parsed["day"]
                days = 1 if day == "today" else (2 if day == "tomorrow" else 3)
                vis = "Temperature"
                if parsed["condition"] in ["rain", "snow"]:
                    vis = "Both"
                question_box.value = ""
            else:
                location = location_box.value.strip() or "London"
                days = days_slider.value
                vis = vis_toggle.value

            units = units_dropdown.value
            weather_data = get_weather_data(location, days, units)

            if vis in ["Temperature", "Both"]:
                create_temperature_plot(weather_data)
            if vis in ["Precipitation", "Both"]:
                create_precipitation_plot(weather_data)

        except Exception as e:
            print(f"❌ Error: {e}")

go_button.on_click(on_click)

# ✅ Click handler for Ask AI button
def on_ask_ai_click(b):
    with output:
        clear_output()
        question = question_box.value.strip()
        if not question:
            print("⚠️ Please enter a question in the 'Ask AI' box.")
            return
        try:
            parsed = parse_weather_question(question)
            response = generate_weather_response(parsed)
            print(response)
            question_box.value = ""
        except Exception as e:
            print(f"❌ Error processing AI question: {e}")

ask_ai_button.on_click(on_ask_ai_click)

# ✅ Display UI with Ask AI button next to question box
print("☁️ Welcome to the AI-Powered Weather Assistant!")
display(widgets.VBox([
    widgets.HBox([question_box, ask_ai_button]),
    widgets.HBox([location_box, days_slider]),
    widgets.HBox([units_dropdown, vis_toggle]),
    go_button,
    output
]))


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.4 MB/s eta 0:00:00
☁️ Welcome to the AI-Powered Weather Assistant!
